In [ ]:
%run -i 'modules/get_securities_codes.py'
%run -i 'modules/get_flattened_codes_new.py'
flattened_codes=get_flattened_codes(0,3,'pickle/oo.pickle')
#ここまでが共通
%run -i 'modules/get_closing_url.py'
%run -i 'modules/flatclosing_urls.py'
flatclosing_urls=save_closing_urls(flattened_codes,0, 3, 'pickle/uu.pickle')

%run -i 'modules/main_data.py'
focus_url, all_title, all_text=main_data(flatclosing_urls, 1, 5)

text1_stringso = [[h2 for h2 in text] for text in all_text]
target_h2_stringso = [[t for t in ext if "売上高" in t or"EPS" in t or "営業利益率" in t or "自己資本比率" in t or
                      "営業活動によるCF" in t or "現金等" in t or "一株配当" in t or "配当性向" in t] for ext in text1_stringso ]
%run -i 'modules/extract_data.py'
test_lists, titles, false = extract_data(target_h2_stringso, all_title)


df_false = pd.DataFrame(false, columns=['title','uu'])
%run -i 'modules/convert_sales_figure.py'
# d_list=[]
sales_list=[[sale.text for sale in test_list[0]]for test_list in test_lists]
#zfill完了
eval_sales_list = [[convert_sales_figure(sale) for sale in sale_list] for sale_list in sales_list]
%run -i 'modules/check_uriagedaka.py'
#採点完了
uriagedaka_stock = [check_uriagedaka(sale) for sale in eval_sales_list]

#-------------------------------------------------------------------------2:ここからEPS2023/01/05--------------------------------------------------------------------------#
eps_lists=[[ep.text for ep in test_list[1]]for test_list in test_lists]
#zfill完了
%run -i 'modules/convert_eps.py'
zeps_list = [[convert_eps(ep) for ep in eps_list]for eps_list in eps_lists]
%run -i 'modules/check_eps.py'
eps_stock = [check_eps(zeps) for zeps in zeps_list]

#-------------------------------------------------------------------------3:ここから営業利益率2023/01/05- -------------------------------------------------------------------------#     

op_lists=[[op.text for op in test_list[2]]for test_list in test_lists]
zop_list = [[eval(s.split('%')[0]) for s in op_list]for op_list in op_lists] # '15.65' のような文字列を取得
%run -i 'modules/check_eiri.py'
eiri_stock = [check_eiri(zop)for zop in zop_list]


#-------------------------------------------------------------------------4:ここから自己資本利益率2023/01/05-------------------------------------------------------------------------#

operation_lists=[[operation.text for operation in test_list[3]]for test_list in test_lists]
%run -i 'modules/floatparcent_hennkan.py'
zoption_list = [[floatparcent_hennkan(s) for s in operation_list]for operation_list in operation_lists]
%run -i 'modules/check_zikosihon.py'
zikosihon_stock = [check_zikosihon(zoption)for zoption in zoption_list]


#-------------------------------------------------------------------------5:ここから現金2023/01/05-------------------------------------------------------------------------#
cash_lists=[[cash.text for cash in test_list[5]]for test_list in test_lists]

#zfill完了
%run -i 'modules/convert_sales_figure.py'
eval_cash_list = [[convert_sales_figure(cash)for cash in cash_list]for cash_list in cash_lists]
%run -i 'modules/check_genkin.py'
genkin_stock = [check_genkin(eval_cash)for eval_cash in eval_cash_list]
#-------------------------------------------------------------------------6:ここからCF -------------------------------------------------------------------------#
cf_flow_lists=[[cfflow.text for cfflow in test_list[4]]for test_list in test_lists]
#zfill完了
%run -i 'modules/convert_flow_figure.py'
eval_cf_flow_list = [[convert_flow_figure(flow)for flow in cf_flow_list]for cf_flow_list in cf_flow_lists]
# モジュール内の関数を呼び出す
%run -i 'modules/check_cflow_order.py'
cflow_stock = [check_cflow_order(eval_cf)for eval_cf in eval_cf_flow_list]

#-------------------------------------------------------------------------7:ここから一株当たり-------------------------------------------------------------------------#
#zfill完了
stock_lists=[[stock.text for stock in test_list[6]]for test_list in test_lists]
%run -i 'modules/convert_stock_figure.py'
zstock_list = [[convert_stock_figure(s) for s in stock_list]for stock_list in stock_lists]
%run -i 'modules/check_zstock_order.py'
hitokabu_stock=[check_zstock_order(zstock)for zstock in zstock_list]

#-------------------------------------------------------------------------8:配当性向-------------------------------------------------------------------------#
return_lists=[[return_money.text for return_money in test_list[7]]for test_list in test_lists]
#zfill完了
%run -i 'modules/convert_return.py'
zreturn_list = [[convert_return(return_money)for return_money in return_list]for return_list in return_lists]
%run -i 'modules/check_zreturn_range.py'
seikou_stock=[check_zreturn_range(zreturn)for zreturn in zreturn_list]
#-------------------------------------------------------------------------ひとまず終了------------------------------------------------------------------------#
analyzes_dict = {}
for i in range(len(titles)):
    analyzes_dict[titles[i]] = [uriagedaka_stock[i], eps_stock[i], eiri_stock[i], zikosihon_stock[i], genkin_stock[i], cflow_stock[i], hitokabu_stock[i],seikou_stock[i]]

gooo = [[key, len(va)] for key, value in analyzes_dict.items() for va in value]
gooo_dict = {}
for lst in gooo:
    key, val = lst
    if key not in gooo_dict:
        gooo_dict[key] = []
    gooo_dict[key].append(val)
for key, value in gooo_dict.items():
    gooo_dict[key] = list(map(lambda x: '○' if x <= 6 else x, value))

ed_list = {}
for key, value in gooo_dict.items():
    if value.count('○') >= 4:
        ed_list[key] = value
print(ed_list)
ed_df = pd.DataFrame.from_dict(ed_list, orient='index', columns=['売上高','EPS','営業利益率','自己資本比率','現金','営業キャッシュフロー','一株配当','配当性向'])
ed_df.style.set_properties(**{'text-align': 'center'})
ed_df.to_csv('csv/stock_target.csv',index=True,encoding='utf-8-sig',mode='a')


time: 4.822501183 [sec]
time: 6.324223042 [sec]
time: 4.419573784 [sec]
time: 7.448007345 [sec]
time: 4.751632214 [sec]
time: 7.035418987 [sec]
time: 3.906721115 [sec]
